<a href="https://colab.research.google.com/github/hpbowala/Language-Modelling-1-/blob/main/makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
# build a vocabulary of chars from dataset 
# we need unique chars that is used in words, and sort them
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}

In [ ]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
X.shape

torch.Size([32, 3])

In [ ]:
e = F.one_hot(X, num_classes=27).float()

In [ ]:
e.shape

torch.Size([32, 3, 27])

In [ ]:
e[0].shape

torch.Size([3, 27])

In [ ]:
e[0]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
# [[487.9,780.5],[487.9,780.5],[487.9,780.5]]
[3, 27] , [27,2]

([3, 27], [27, 2])

In [ ]:
g = torch.Generator().manual_seed(265989)
W_binning = torch.randn((27,2),generator=g)
W_binning

tensor([[-0.3412, -0.5439],
        [-0.4370, -2.1848],
        [-1.7497,  0.4085],
        [ 0.3157,  0.8403],
        [ 0.8791, -0.2559],
        [-1.5511,  0.0979],
        [ 0.7870, -0.7681],
        [-0.1608, -0.4183],
        [ 0.8205, -0.0401],
        [ 0.3133, -0.4411],
        [-0.2562,  0.6405],
        [ 0.1609,  1.2027],
        [ 0.0508, -0.1308],
        [ 1.0570, -0.1836],
        [-2.0756,  0.8777],
        [ 0.2965,  0.1616],
        [ 1.8882,  0.5500],
        [ 0.2564,  0.7203],
        [ 0.2077, -1.5529],
        [ 0.8031, -0.4730],
        [ 0.3695, -0.3791],
        [ 0.0861, -0.1276],
        [-0.6042, -0.7568],
        [-1.4354, -1.2370],
        [ 0.1461, -0.4351],
        [ 0.5395, -1.1079],
        [ 0.0142,  0.6782]])

In [ ]:
e[3] @ W_binning

tensor([[-1.5511,  0.0979],
        [ 1.0570, -0.1836],
        [ 1.0570, -0.1836]])

In [ ]:
X[3]

tensor([ 5, 13, 13])

In [ ]:
W_binning[X].shape

torch.Size([32, 3, 2])

In [ ]:
# binning the values in the first layer to two dimensions
out_binned = W_binning[X]
out_binned.shape

torch.Size([32, 3, 2])

In [ ]:
out_binned[0]

tensor([[-0.3412, -0.5439],
        [-0.3412, -0.5439],
        [-0.3412, -0.5439]])

In [ ]:
W_hidden = torch.randn((6,100),generator=g)
bias = torch.randn((100),generator=g)
concat_out_binned = torch.cat((out_binned[:,0,:],out_binned[:,1,:],out_binned[:,2,:]),1)
# concat_out_binned.shape
in_hidden_activation = concat_out_binned @ W_hidden + bias
out_hidden = torch.tanh(in_hidden_activation)
out_hidden.shape

torch.Size([32, 100])